#  Consignes

## Description

Ouvrir le fichier ks-projects-201801.csv, il recense environ 100 000 projets KickStarter. Intégrer les données directement avec L'API Python dans une base de données Mongo. 

Il conviendra de bien spécifier manuellement l'ID du document. Pensez aussi à bien formatter le type des données pour profiter des méthodes implémentées par Mongo. L'ensemble de données n'est pas forcément nécessaire, c'est à vous de créer votre modèle de données.

## Questions

- 1) Récupérer les 5 projets ayant reçu le plus de promesse de dons.
- 2) Compter le nombre de projets ayant atteint leur but.
- 3) Compter le nombre de projets pour chaque catégorie.
- 4) Compter le nombre de projets français ayant été instanciés avant 2016.
- 5) Récupérer les projets américains ayant demandé plus de 200 000 dollars.
- 6) Compter le nombre de projet ayant "Sport" dans leur nom

In [1]:
import pandas as pd
import pymongo

In [2]:
client = pymongo.MongoClient()
database = client['exercices']
collection = database['kickstarter']

In [3]:
df_ks = pd.read_csv("./data/ks-projects-201801-sample.csv")
df_ks.head()

c:\users\nicol\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (6,8,10,12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000,2011-08-17 06:31:31,1145,canceled,24,US,1145,1145.000000
1,1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000,2012-07-23 20:46:48,1851,failed,28,US,1851,1851.000000
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000,2016-07-01 21:55:54,7534,successful,254,US,3796,7534.000000
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000,2017-08-28 15:47:02,16298,successful,367,US,2670,16298.000000
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000,2016-01-25 17:37:10,1,failed,1,CA,0.708148,0.738225


Ce warning intervient lorsque pandas n'arrive pas à inférer le type de données. Il est sympa il précise les colones 6,8,10,12. 

In [4]:
df_ks.columns[[6,8,10,12]]

Index(['goal', 'pledged', 'backers', 'usd pledged'], dtype='object')

## Question 0

In [5]:
df_ks.dtypes

ID                    int64
name                 object
category             object
main_category        object
currency             object
deadline             object
goal                 object
launched             object
pledged              object
state                object
backers              object
country              object
usd pledged          object
usd_pledged_real    float64
dtype: object

In [56]:
#On supprime les colonnes

#On modifie
df_ks["launched"] = df_ks["launched"].str[:4]
df_ks = df_ks.rename({"usd pledged":"usd_pledged"}, axis='columns')

df_ks['pledged'] = df_ks['pledged'].astype('float64')
df_ks['state'] = df_ks['state'].astype('string')
df_ks['name'] = df_ks['name'].astype('string')

df_ks = df_ks.dropna()

### Netoyer les données

### Importer les données

In [57]:
dict_ks = df_ks.to_dict(orient='records')

In [58]:
collection.insert_many(dict_ks)

## Question 1  

In [59]:
#Récupérer les 5 projets ayant reçu le plus de promesse de dons.
plusdon = collection.find().sort([("pledged",-1)]).limit(5)
list(plusdon)

[{'_id': ObjectId('60210c8d76ed13ebdcfbfbd9'),
  'name': "COOLEST COOLER: 21st Century Cooler that's Actually Cooler",
  'category': 'Product Design',
  'main_category': 'Design',
  'goal': '50000.0',
  'launched': '2014',
  'pledged': 13285226.36,
  'state': 'successful',
  'country': 'US',
  'usd_pledged': '13285226.36',
  'usd_pledged_real': 13285226.36},
 {'_id': ObjectId('60210c4e76ed13ebdcf76b02'),
  'name': "COOLEST COOLER: 21st Century Cooler that's Actually Cooler",
  'category': 'Product Design',
  'main_category': 'Design',
  'goal': '50000.0',
  'launched': '2014',
  'pledged': 13285226.36,
  'state': 'successful',
  'country': 'US',
  'usd_pledged': '13285226.36',
  'usd_pledged_real': 13285226.36},
 {'_id': ObjectId('6021108176ed13ebdc00841f'),
  'name': "COOLEST COOLER: 21st Century Cooler that's Actually Cooler",
  'category': 'Product Design',
  'main_category': 'Design',
  'goal': '50000.0',
  'launched': '2014',
  'pledged': 13285226.36,
  'state': 'successful',
  'c

## Question 2

In [60]:
#Compter le nombre de projets ayant atteint leur but.
succes = collection.aggregate([{"$group":{"_id":"$state","number":{"$sum":1}}}])
list(succes)

[{'_id': 'successful', 'number': 229644},
 {'_id': 'suspended', 'number': 3234},
 {'_id': 'live', 'number': 4908},
 {'_id': 'failed', 'number': 339523},
 {'_id': 'undefined', 'number': 1846},
 {'_id': 'canceled', 'number': 66390}]

## Question 3

In [61]:
#Compter le nombre de projets pour chaque catégorie.
nombrecate = collection.aggregate([{"$group":{"_id":"$category","total":{"$sum":1}}}])
list(nombrecate)

[{'_id': 'Musical', 'total': 1599},
 {'_id': 'Fantasy', 'total': 578},
 {'_id': 'Video Games', 'total': 20893},
 {'_id': 'Performance Art', 'total': 3733},
 {'_id': 'Animals', 'total': 409},
 {'_id': 'Small Batch', 'total': 3057},
 {'_id': 'Interactive Design', 'total': 626},
 {'_id': 'Periodicals', 'total': 2223},
 {'_id': 'Embroidery', 'total': 209},
 {'_id': 'Rock', 'total': 11651},
 {'_id': 'Wearables', 'total': 2208},
 {'_id': 'Zines', 'total': 600},
 {'_id': "Farmer's Markets", 'total': 783},
 {'_id': 'Kids', 'total': 469},
 {'_id': 'Photobooks', 'total': 2656},
 {'_id': 'Action', 'total': 1246},
 {'_id': 'Letterpress', 'total': 114},
 {'_id': 'Technology', 'total': 11661},
 {'_id': 'Calendars', 'total': 486},
 {'_id': 'Radio & Podcasts', 'total': 1513},
 {'_id': 'Textiles', 'total': 457},
 {'_id': 'Indie Rock', 'total': 9450},
 {'_id': 'Chiptune', 'total': 46},
 {'_id': 'Print', 'total': 1269},
 {'_id': 'Punk', 'total': 535},
 {'_id': 'Animation', 'total': 4439},
 {'_id': 'Quilt

## Question 4

## Question 5

In [62]:
#Récupérer les projets américains ayant demandé plus de 200 000 dollars.
expensive=collection.find({"$and":[{"Country":"US"},{"Goal":{"$gte":200000}}]})
list(expensive)

[]

## Question 6 

In [64]:
#Compter le nombre de projet ayant "Sport" dans leur nom
Sport=collection.find({"name":{'$regex':"Sport"}})
len(list(Sport))

1385